In [2]:
%pip install  cvzone

Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip show cvzone


Name: cvzone
Version: 1.6.1
Summary: Computer Vision Helping Library
Home-page: https://github.com/cvzone/cvzone.git
Author: Computer Vision Zone
Author-email: contact@computervision.zone
License: MIT
Location: /Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages
Requires: numpy, opencv-python
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install mediapipe

Note: you may need to restart the kernel to use updated packages.


In [1]:
import cv2
import csv
import cvzone
from cvzone.HandTrackingModule import HandDetector
import time #the reason we imported time is because we don't want the click to be repeated multiple times in the console

cap = cv2.VideoCapture(0)
cap.set(3, 1280) #width
cap.set(4, 720) #highet
detector = HandDetector(detectionCon=0.8)


class MCQ(): #create an mcq class where every question is an object
    def __init__(self, data): #the data for each question
        self.question = data[0]
        self.choice1 = data[1]
        self.choice2 = data[2]
        self.choice3 = data[3]
        self.choice4 = data[4]
        self.answer = int(data[5]) #the value of this is an integer now because we'll this value (the true value in the csv file) with the value the user choose

        self.userAns = None #as a start before the user chooses anything

    def update(self, cursor, bboxs):

        for x, bbox in enumerate(bboxs):
            x1, y1, x2, y2 = bbox
            if x1 < cursor[0] < x2 and y1 < cursor[1] < y2: #this checks that the curser (in our program the tip of the index finger) is on a specific box or not
                #0 refers to x and 1 refers to y
                self.userAns = x + 1 #the reason we're using a +1 is becuase in the csv file we started counting from a and not 0, otherwise we would've putten it without the plus
                cv2.rectangle(img, (x1, y1), (x2, y2), (0,255,0), cv2.FILLED) #the numbers refers to the color green, the failed to the thickness


# Import csv file data
pathCSV = "CSV_file" #name of the file or you can use it directly 
with open(pathCSV, newline='\n') as f: 
    #(with) is a context manager in Python that is used here to manage the opening and closing of the file
    #(open) function in Python used to open a file
    #(newline='\n'), you're telling Python to use \n as the line separator in the file. this is related to the csv file
    #(f) This part of the statement assigns the file object returned by open to the variable f
    reader = csv.reader(f) #creates an object that operates like an iterator to read data from a CSV file line by line
    dataAll = list(reader)[1:] #reader is converted into a list that contains the data for all the question, the heading is ignored in the output becuase we started from 1 instead of 0

# Create Object for each MCQ
mcqList = []
for q in dataAll:
    mcqList.append(MCQ(q))

print("Total MCQ Objects Created:", len(mcqList)) #num of questions

qNo = 0
qTotal = len(dataAll)

while True:
    success, img = cap.read()
    img = cv2.flip(img, 1)
    hands, img = detector.findHands(img, flipType=False)

    if qNo < qTotal:
        mcq = mcqList[qNo]
        #each both refer to a choice in a question 
        #the second parameter is the text of our object
        img, bbox = cvzone.putTextRect(img, mcq.question, [100, 100], 2, 2, offset=50, border=5)#position,scale
        img, bbox1 = cvzone.putTextRect(img, mcq.choice1, [100, 250], 2, 2, offset=50, border=5)#offset provide margins and clearance
        img, bbox2 = cvzone.putTextRect(img, mcq.choice2, [400, 250], 2, 2, offset=50, border=5)
        img, bbox3 = cvzone.putTextRect(img, mcq.choice3, [100, 400], 2, 2, offset=50, border=5)
        img, bbox4 = cvzone.putTextRect(img, mcq.choice4, [400, 400], 2, 2, offset=50, border=5)

        if hands: #this checks if we have clicked on an answer or not 
            lmList = hands[0]['lmList'] #this is a variable 
            cursor = lmList[8] #the curser refered to here is our index finger (mediapipe refers to the tip of the index finger as 8)

            # Capture all returned values in a tuple
            results = detector.findDistance(lmList[8][:2], lmList[12][:2]) #results is type tuple that takes all information about 2 landmarks
            #this takes the x1,y1,x2,y2 coordinates (distance)
            #this provides us with the distance between the middle finger and the index finger
            #the reason we do this is because if the distance is low then it means we clicked
            length = results[0] #type is float (it takes the first arguemnt of results which is the distance)
            
            if length < 35: #this distance means that the two fingers and right next to eachother
                #we'll use the (update method) which we created to determine which choice has been clicked
                mcq.update(cursor, [bbox1, bbox2, bbox3, bbox4]) #it has to check all the options in each question 
                if mcq.userAns is not None: #this checks if something is being clicked
                    time.sleep(0.3) #so that it only catches the click once 
                    qNo += 1 # this moves to the next question (the above list will be updated as well)
    else:
        score = 0
        for mcq in mcqList:
            if mcq.answer == mcq.userAns: #here we're refering to the answer in the csv file to the one in the msq object
                score += 1 #if the if above was flase we'll just ignore the choice and score will not be incremented
        score = round((score / qTotal) * 100, 2) #gives the score as percentage (it's orginally the number of questions that was correctly answered)
        img, _ = cvzone.putTextRect(img, "Quiz Completed", [250, 300], 2, 2, offset=50, border=5)
        img, _ = cvzone.putTextRect(img, f'Your Score: {score}%', [700, 300], 2, 2, offset=50, border=5)

    # Draw Progress Bar
    barValue = 150 + (950 // qTotal) * qNo #150 is the starting point #950 is the total width of the bar (150-1100)
    cv2.rectangle(img, (150, 600), (barValue, 650), (0,255,0), cv2.FILLED)
    cv2.rectangle(img, (150, 600), (1100, 650), (0,255,0), 5)
    img, _ = cvzone.putTextRect(img, f'{round((qNo / qTotal) * 100)}%', [1130, 635], 2, 2, offset=16) #this simply provides a percenatge of the progress

    cv2.imshow("Img", img)
    cv2.waitKey(1)


I0000 00:00:1707336520.491688       1 gl_context.cc:344] GL version: 2.1 (2.1 INTEL-20.6.4), renderer: Intel(R) Iris(TM) Plus Graphics 645
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


Total MCQ Objects Created: 4
<class 'tuple'>
<class 'float'>
<class 'tuple'>
<class 'float'>
<class 'tuple'>
<class 'float'>
<class 'tuple'>
<class 'float'>
<class 'tuple'>
<class 'float'>
<class 'tuple'>
<class 'float'>
<class 'tuple'>
<class 'float'>
<class 'tuple'>
<class 'float'>
<class 'tuple'>
<class 'float'>
<class 'tuple'>
<class 'float'>
<class 'tuple'>
<class 'float'>
<class 'tuple'>
<class 'float'>
<class 'tuple'>
<class 'float'>
<class 'tuple'>
<class 'float'>
<class 'tuple'>
<class 'float'>
<class 'tuple'>
<class 'float'>
<class 'tuple'>
<class 'float'>
<class 'tuple'>
<class 'float'>
<class 'tuple'>
<class 'float'>
<class 'tuple'>
<class 'float'>
<class 'tuple'>
<class 'float'>
<class 'tuple'>
<class 'float'>
<class 'tuple'>
<class 'float'>


: 

: 